In [1]:
#Run with python3

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import datetime
import os
import string
import re

In [2]:
def fetch(url):
    req=Request(url, headers={'User-Agent': 'Misc Robot'})
    html_week = urlopen(req).read()
    soup = BeautifulSoup(html_week, 'html.parser')
    return soup
        
def get_num(text):
    return re.findall(r"[0-9]*[0-9]", text)[0]


In [ ]:
#[str(x) for x in reversed(range(2,10))]

In [28]:
class Catalog:
    def __init__(self):
        self.base_url='http://usa.newonnetflix.info/catalog/a2z/all'
        self.pages=list(string.ascii_lowercase)
        self.object_urls=[]
        
    def fetch_all(self):
        for page in self.pages:
            url=self.base_url+'/'+page
            print(url)
            soup = fetch(url)
            self.parse(soup)
            print(len(self.object_urls))
            
    def parse_film_number(self, soup):
        total=soup.find('section').find_all('em')[0].text
        return get_num(total)

    def parse_category_number(self, soup):
        total=soup.find('section').find_all('em')[1].text
        return get_num(total)
    
    def parse(self, soup):
        #self.num_objects=self.parse_category_number(soup)
        articles=soup.find_all('section')[3].find_all('article')
        for article in articles:
            self.object_urls.append(article.find_all('a')[1]['href'])       
        
    def __repr__(self):
        return self.base_url

In [29]:
netflix=Catalog()
netflix.fetch_all()

http://usa.newonnetflix.info/catalog/a2z/all/a
http://usa.newonnetflix.info/catalog/a2z/all/b
http://usa.newonnetflix.info/catalog/a2z/all/c
http://usa.newonnetflix.info/catalog/a2z/all/d
http://usa.newonnetflix.info/catalog/a2z/all/e
http://usa.newonnetflix.info/catalog/a2z/all/f
http://usa.newonnetflix.info/catalog/a2z/all/g
http://usa.newonnetflix.info/catalog/a2z/all/h
http://usa.newonnetflix.info/catalog/a2z/all/i
http://usa.newonnetflix.info/catalog/a2z/all/j
http://usa.newonnetflix.info/catalog/a2z/all/k
http://usa.newonnetflix.info/catalog/a2z/all/l
http://usa.newonnetflix.info/catalog/a2z/all/m
http://usa.newonnetflix.info/catalog/a2z/all/n
http://usa.newonnetflix.info/catalog/a2z/all/o
http://usa.newonnetflix.info/catalog/a2z/all/p
http://usa.newonnetflix.info/catalog/a2z/all/q
http://usa.newonnetflix.info/catalog/a2z/all/r
http://usa.newonnetflix.info/catalog/a2z/all/s
http://usa.newonnetflix.info/catalog/a2z/all/t
http://usa.newonnetflix.info/catalog/a2z/all/u
http://usa.ne

In [30]:
import pickle as pkl

In [31]:
with open('object_urls.pkl', 'wb') as f:
    pkl.dump(netflix.object_urls, f)

In [32]:
class Article():
    def __init__(self, url):
        self.url='http://usa.newonnetflix.info'+url
        self.build()
    
    def build(self):
        soup=fetch(self.url)
        self.parse(soup)
        return self.attrs
    
    def parse(self, soup):
        article=soup.find_all('article')[0]
        attrs={}
        attrs['img_src']=article.find_all('img')[0]['src']
        attrs['genre']=article.select('div[class=genre]')[0].text
        
        p=article.find_all('p')

        
        for i in range(0,len(p)):
            if p[i].find('strong'):


                attr=p[i].find('strong').text[:-1]
                if attr not in ['IMDB', 'IMDB Rating', 'Netflix Rating', 'Description']:
                    try:
                        val=p[i].text.split(': ')[1]
                    except:
                        print(attr, self.url)
                elif attr=='Description':
                    val=p[i+1].text

                elif attr=='IMDB':
                    attr='imdb_url'
                    val=p[i].find('a')['href']

                elif attr in ['Netflix Rating', 'IMDB Rating']:
                    try:
                        val=p[i].find_all('img')[-1]['title'].split(' ')[2]
                    except:
                        print(attr, self.url)

                attrs[attr.lower()]=val
                
        self.attrs=attrs

In [42]:
def build(url):
    soup=fetch('http://usa.newonnetflix.info'+url)
    return parse(soup)

def parse(soup):
    article=soup.find_all('article')[0]
    attrs={}
    attrs['img_src']=article.find_all('img')[0]['src']
    attrs['genre']=article.select('div[class=genre]')[0].text
    attrs['title']=article.select('h1')[0].text

    p=article.find_all('p')


    for i in range(0,len(p)):
        if p[i].find('strong'):


            attr=p[i].find('strong').text[:-1]
            if attr not in ['IMDB', 'IMDB Rating', 'Netflix Rating', 'Description']:
                try:
                    val=p[i].text.split(': ')[1]
                except:
                    print(attr)
            elif attr=='Description':
                val=p[i+1].text

            elif attr=='IMDB':
                attr='imdb_url'
                val=p[i].find('a')['href']

            elif attr in ['Netflix Rating', 'IMDB Rating']:
                try:
                    val=p[i].find_all('img')[-1]['title'].split(' ')[2]
                except:
                    print(attr)

            attrs[attr.lower()]=val
    return attrs

In [44]:
from multiprocessing.dummy import Pool  # This is a thread-based Pool
from multiprocessing import cpu_count
pool = Pool(processes=cpu_count() * 2)  # Creates a Pool with cpu_count * 2 threads.
media=pool.map(build, netflix.object_urls)

History
History
History
History
History
History
History
History
History
IMDB Rating
History
History
History
History
History
IMDB Rating
IMDB Rating
History
History
IMDB Rating
IMDB Rating
IMDB Rating
History
History
History
IMDB Rating
History
IMDB Rating
History
History
History
History
History
IMDB Rating
History
History
History
History
History
History
History
History
History
History
History
IMDB Rating
History
IMDB Rating
History
IMDB Rating
IMDB Rating
History
History
History
History
History
History
IMDB Rating
IMDB Rating
History
History
History
IMDB Rating
History
History
History
History
IMDB Rating
IMDB Rating
History
History
History
History
History
IMDB Rating
History
IMDB Rating
History
History
History
History
History
History
History
IMDB Rating
History
History
IMDB Rating
History
History
History
History
IMDB Rating
History
HistoryHistory

History
History
History
History
History
History
History
History
IMDB Rating
History
History
History
History
History
History
History
History


In [40]:
import pandas as pd

In [45]:
media=pd.DataFrame.from_records(media)
media=media[['cast', 'title',
       'certificate', 'date added', 'description', 'director', 'duration',
       'episode length', 'genre',  'imdb rating', 'imdb_url',
       'img_src', 'netflix rating','year']]

In [46]:
media.head()

,cast,title,certificate,date added,description,director,duration,episode length,genre,imdb rating,imdb_url,img_src,netflix rating,year
0,Misty Copeland,A Ballerina's Tale,NR,2nd February 2016,This documentary charts the inspiring ascent o...,Nelson George,1hr 23m,NaN,"Biographical Documentaries, Social & Cultural ...",6.4/10,http://www.imdb.com/title/tt4504040/,https://art-s.nflximg.net/baafc/333ab0fb230639...,4.2/5,2015
1,"Yaël Abecassis, Ali Suliman, Marlene Bajjali, ...",A Borrowed Identity,NR,27th October 2015,A promising Arab student brought up in Israel ...,Eran Riklis,1hr 44m,NaN,"Dramas based on a book, Dramas, International ...",7.3/10,http://www.imdb.com/title/tt2841572/,https://art-s.nflximg.net/02cc2/e1b23c0efccd17...,3.7/5,2014
2,Ashleigh Ball,A Brony Tale,NR,21st October 2014,Cartoon voice actor Ashleigh Ball reveals the ...,Brent Hodge,1hr 18m,NaN,"Social & Cultural Documentaries, Documentaries...",5.6/10,http://www.imdb.com/title/tt3219678/,https://art-s.nflximg.net/ac807/212c75a6b5d504...,3.2/5,2014
3,"Sam Rockwell, Vince Vaughn, Evan Rachel Wood, ...",A Case of You,R,5th December 2015,Nursing a secret crush on a cute coffee barist...,Kat Coiro,1hr 31m,NaN,"Romantic Comedies, Comedies, Independent Films",5.6/10,http://www.imdb.com/title/tt1817081/,https://art-s.nflximg.net/6bb9a/b1f32926ed59ee...,3.3/5,2013
4,"Marcia Gay Harden, Anjelica Huston, Matthew Mo...",A Cat in Paris,PG,Unknown Date,When Zoe discovers that her cat is also a thie...,"Jean-Loup Felicioli, Alain Gagnol",1hr 4m,NaN,"Family Adventures, Crime Dramas, Family Featur...",7/10,http://www.imdb.com/title/tt1673702/,https://art-s.nflximg.net/5120a/ac7353edffe929...,3.8/5,2010


In [47]:
media.to_csv('full-media.csv')

In [ ]:
#add mapreduce